In [89]:
f = """
def foo():
    for i in range(2):
        print 1
        print 3
    print 2
    print 4
"""

In [223]:
def blobify(f):
    blobs = []
    stack = ()
    stack_size = 0

    for line in f.splitlines():
        tab = line[:len(line) - len(line.lstrip())]
        if tab:
            break
    else:
        tab = ' '

    for line in f.splitlines():
        line = line.rstrip()

        if not line:
            continue

        tabs = 0
        while line[tabs * len(tab):(tabs+1) * len(tab)] == tab:
            tabs += 1
        
        for i in range(stack_size - tabs):
            stack, item1 = stack
            blobs.append(item1)
            if len(stack) == 2:
                stack, item2 = stack
                stack = stack, (item2, item1)
            stack_size -= 1

        stack = stack, line.strip()
        stack_size += 1

    while len(stack) == 2:
        stack, item1 = stack
        blobs.append(item1)
        if len(stack) == 2:
            stack, item2 = stack
            stack = stack, (item2, item1)
        
    return blobs

def blob_len(blob):
    l = 1
    while len(blob) == 2:
        blob, _ = blob
        l += 1
    return l

def blob_size(blob):
    s = 0
    while len(blob) == 2:
        blob, item = blob
        if isinstance(item, tuple):
            blob = (blob, item[0]), item[1]
        else:
            s += 1
    return s

def blob_to_txt(blob, tabs=0):
    if isinstance(blob, str) or isinstance(blob, unicode):
        return '    '*tabs + blob, tabs + 1
    elif len(blob) == 2:
        s, tabs = blob_to_txt(blob[0], tabs)
        return '%s\n%s' % (s, blob_to_txt(blob[1], tabs)[0]), tabs

def print_blob(blob):
    print blob_to_txt(blob, 0)[0]

In [226]:
blob_size(blobify(f)[-1])

5

In [69]:
import sys

In [155]:
blobs = collections.Counter()

for dirp in sys.path:
    for dirpath, dirnames, filenames in os.walk(dirp):
        for filename in filenames:
            if filename.endswith('.py'):
                try:
                    new_blobs = blobify(open(os.path.join(dirpath, filename), 'r').read())
                    blobs.update([b for b in new_blobs if 20 >= blob_len(b) >= 4])
                except UnicodeDecodeError:
                    pass

In [163]:
import ast

In [308]:
class RewriteName(ast.NodeTransformer):
    def visit_Name(self, node):
        return ast.copy_location(ast.Name(id='var', ctx=ast.Load()), node)
    
    def visit_FunctionDef(self, node):
        self.generic_visit(node)
        return ast.copy_location(ast.FunctionDef(name='func', args=node.args, body=node.body, ctx=ast.Load()), node)
    
    def visit_Attribute(self, node):
        self.generic_visit(node)
        return ast.copy_location(ast.Attribute(attr='attr', value=node.value, ctx=ast.Load()), node)
    
    def visit_Num(self, node):
        return ast.copy_location(ast.Num(n=0), node)
    
    def visit_Str(self, node):
        return ast.copy_location(ast.Str(s="str"), node)

In [307]:
ast.dump(RewriteName().visit(ast.parse('x="1"')), annotate_fields=False)

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_attributes', '_fields', 'col_offset', 'lineno', 's']


"Module([Assign([Name('var', Load())], Str('str'))])"

In [319]:
asts = collections.defaultdict(set)
for b in blobs:
    if 5 <= blob_size(b) < 20:
        try:
            asts[ast.dump(RewriteName().visit(ast.parse(blob_to_txt(b)[0])), annotate_fields=False)].add(b)
        except SyntaxError:
            pass

In [320]:
sorted(set(map(len, asts.values())))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 52,
 56,
 59,
 68,
 73,
 90,
 91,
 93,
 128,
 130,
 140,
 153,
 171,
 219,
 229,
 327,
 466]

In [331]:
for _, v, k in sorted(((len(v), v, k) for k, v in asts.items()))[-20:-10][::-1]:
    print k, len(v)
    for s in list(v)[:10]:
        print_blob(s)
        print
    print
        

Module([FunctionDef('func', arguments([Name('var', Load()), Name('var', Load())], None, None, []), [Expr(Str('str'))])]) 91
def putChild(path, child):
    """
    Put a child IResource implementor at the given path.
    @param path: A single path component, to be interpreted relative to the
        path this resource is found at, at which to put the given child.
        For example, if resource A can be found at I{http://example.com/foo}
        then a call like C{A.putChild(b"bar", B)} will make resource B
        available at I{http://example.com/foo/bar}.
    @type path: C{bytes}
    """

def toString(self, inObject):
    """
    Convert a Python object into a string for passing over the network.
    @param inObject: an object of the type that this Argument is intended
    to deal with.
    @return: the wire encoding of inObject
    @rtype: C{bytes}
    """

def setHeader(k, v):
    """
    Set an HTTP response header.  Overrides any previously set values for
    this header.
    @t